In [1]:
#Library 예시

import math
import time
from itertools import chain
import argparse
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
# 모델 학습을 위해 CUDA 환경 설정. : 지피유 설정
device = torch.device('cuda:1' if torch.cuda.is_available else 'cpu')
#device=torch.device('cpu')
torch.cuda.is_available()

True

In [2]:
u=np.linspace(1000,2000,51)
w=np.linspace(2300,8000,20)
wavenumbers=np.append(u,w)
len(wavenumbers)
wavenumbers=np.linspace(1000, 8000, 71)

dataset=>train, val

In [3]:
# 별도의 데이터 Pre-Processing 과정은 없고 모델 훈련시 검증을 위해 train 중 10000개를 validation 용으로 분리.
# 새로 만든 train.csv는 train1.csv, validation은 val.csv로 저장.
# dataframe.sample(frac=1) 을 통해 셔플.
path_train = 'train_same_layer_level_4per.csv'
#path_test = 'test.csv'
layers = [[str(i) for i in wavenumbers.tolist()],[str(i) for i in wavenumbers.tolist()],['layer(nm)','level(%)'], [str(i) for i in np.arange(1000,8100,100).tolist()]]
layers = list(chain(*layers))

train = pd.read_csv(path_train)
print(train.shape)
train = train.sample(frac=1)
rows, cols = train.shape

train1 = train.iloc[:rows - (int(0.1*len(train))),:]
train1 = train1.values
train1 = pd.DataFrame(data=train1,columns=layers)
train1.to_csv('train1_same_layer_level.csv', index_label='id')

print("train file saved....")
val = train.iloc[rows - (int(0.1*len(train))):,:]
val = val.values
val = pd.DataFrame(data=val,columns=layers)
val.to_csv('val_same_layer_level.csv', index_label='id')

(4824600, 215)
train file saved....


In [4]:
# 새로 만든 train/ val 모델 학습 데이터 경로를 설정.
train_path = 'train1_same_layer_level.csv'#레이어를 같게 두고 풀때
val_path = 'val_same_layer_level.csv'

lr = 1e-03
adam_epsilon = 1e-06
epochs = 100 #12-17: 200->100
batch_size = 2048
warmup_step = 2000

class PandasDataset(Dataset):
    def __init__(self, path):
        super(PandasDataset, self).__init__()
        train = pd.read_csv(path).iloc[:,1:]
        self.train_X, self.train_Y = train.iloc[:,144:], train.iloc[:,0:144]
        self.tmp_x , self.tmp_y = self.train_X.values, self.train_Y.values
    
    def __len__(self):
        return len(self.train_X)

    def __getitem__(self, idx):
        return {
            'X':torch.from_numpy(self.tmp_x)[idx],
            'Y':torch.from_numpy(self.tmp_y)[idx]
        }
            
train_dataset = PandasDataset(train_path)
train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=True, num_workers=0)#card에 data size/batch size만큼 monunt

val_dataset = PandasDataset(val_path)
val_loader = DataLoader(val_dataset, batch_size=batch_size, pin_memory=True,  num_workers=0)

Conv Neural Net

In [19]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        #input: 71X1
        self.conv1=nn.Sequential(nn.Conv1d(1,32,kernel_size=3, stride=2 , padding=1),nn.BatchNorm1d(32),nn.ReLU(),nn.MaxPool1d(1, stride=1, padding=0))
        #36x32: 32-channel 개수
        self.conv2=nn.Sequential(nn.Conv1d(32,64,kernel_size=1,stride=1,padding=0),nn.BatchNorm1d(64),nn.ReLU(),nn.MaxPool1d(1, stride=1))
        #36X64
        self.conv3=nn.Sequential(nn.Conv1d(64,128,kernel_size=1,stride=1,padding=0),nn.BatchNorm1d(128),nn.ReLU(),nn.MaxPool1d(1, stride=1))
        #36x128
        self.conv4=nn.Sequential(nn.Conv1d(128,128,kernel_size=1,stride=1,padding=0),nn.BatchNorm1d(128),nn.ReLU(),nn.MaxPool1d(1, stride=1))
        #36x128
        self.conv5=nn.Sequential(nn.Conv1d(128,64,kernel_size=1,stride=1,padding=0),nn.BatchNorm1d(64),nn.ReLU(),nn.MaxPool1d(1, stride=1))
        #36x64
        self.conv6=nn.Sequential(nn.Conv1d(64,32,kernel_size=1,stride=1,padding=0),nn.BatchNorm1d(32),nn.ReLU(),nn.MaxPool1d(1, stride=1))
        #36X32
        self.fconv1=nn.Sequential(nn.Conv1d(32,32,kernel_size=2,stride=2,padding=0),nn.BatchNorm1d(32),nn.ReLU(),nn.MaxPool1d(1, stride=1))
        #18x32
        self.fc1=nn.Sequential(nn.Linear(18*32, 144, bias=True), nn.BatchNorm1d(144)) #nk out
        #Add layer to fit thick
        self.layer1=nn.Sequential(nn.Linear(18*32,2000), nn.BatchNorm1d(2000), nn.ReLU())
        #2000
        self.layer2=nn.Sequential(nn.Linear(2000,4000),nn.BatchNorm1d(4000), nn.ReLU())
        #4000
        self.layer3=nn.Sequential(nn.Linear(4000,2000), nn.BatchNorm1d(2000), nn.ReLU())
        #2000
        self.layer4=nn.Sequential(nn.Linear(2000,300), nn.BatchNorm1d(300), nn.ReLU())
        #thick and level
        self.fc2=nn.Sequential(nn.Linear(300,2)) #thickness, level out
        
    def forward(self, x):
#         print(x.shape) batch size X columns 
        x=x.view(x.shape[0],1,-1)
#         print(x.shape) batch size X 1 X columns 
        upout1 = self.conv1(x) #batch size X 36 X 32
        
        upout2 = self.conv2(upout1) #36x64
        #print(out.shape)
        upout3 = self.conv3(upout2) #36x128
        
        downout3=self.conv4(upout3) #36x128
        
        downout2=self.conv5(downout3+upout3)#36x64
        
        downout1=self.conv6(downout2+upout2) #36x32
        
        fout=self.fconv1(downout1+upout1) #18x32
        #print(fout.shape)
        
        out=fout.view(x.shape[0], fout.size(1)*fout.size(2))
        #thickness
        #thout=self.layer1(out) #2000
        #print(out.shape)
       # print(thout.shape)
        out1=self.fc1(out)
        #thickness
#         out2=self.layer2(thout)#4000
#         out2=self.layer3(out2)
#         out2=self.layer4(out2)
#         out2=self.fc2(out2)
#         print(out1.shape)
#         print(out2.shape)
        return out1

In [20]:
#model=FCN()
model=ConvNet() #ReLU를 뺐더니 이득봄.
model=model.to(device)

In [21]:
nk_loss_fn=nn.KLDivLoss(reduction='batchmean')
thick_loss_fn=nn.L1Loss()
level_loss_fn=nn.L1Loss()
loss_fn=nn.L1Loss()
loss_fn2=nn.MSELoss()

In [22]:
optimizer = optim.Adam(model.parameters(), lr = lr, eps=adam_epsilon)

total_loss=0.0
n_k_loss=0.0
thick_loss=0.0
level_loss=0.0
total_val_loss=0.0

version = time.localtime()[3:5]
curr_lr = lr

n_val_loss = 10000000. # 가장 낮은 validation loss를 저장하기 위해서 변수 설정.
train_loss_epoch=[]
nk_loss_epoch=[]
thick_loss_epoch=[]
level_loss_epoch=[]
val_loss_epoch=[]

start = time.time() # 학습시간 측정.

for epoch in range(epochs):
    total_loss=0.0
    n_k_loss=0.0
    thick_loss=0.0
    level_loss=0.0
    total_val_loss=0.0
    for i, data in enumerate(tqdm(train_loader, desc='*********Train mode*******')):  # train 데이터를 부르고 학습.
        # forward pass
        pred1 = model(data['X'].float().to(device))
       # print('data[Y].shape: ',data['Y'].shape)
       # print('pred1.shape: ',pred1.shape)
        loss_nk = loss_fn2(pred1[:,:142], data['Y'].float().to(device)[:,:142])
        loss_thick=loss_fn(pred1[:,142], data['Y'].float().to(device)[:,142])
        loss_level=loss_fn(pred1[:,143], data['Y'].float().to(device)[:,143])
        
        loss=torch.sqrt(loss_nk) + loss_thick + loss_level
        
        # backward pass
        optimizer.zero_grad() # optimizer 객체 사용해서 학습 가능한 가중치 변수에 대한 모든 변화도를 0으로 만듦
        loss.backward()
        optimizer.step() # update optimizer params
        
        total_loss+=loss.item()
        n_k_loss += loss_nk.item()
        thick_loss += loss_thick.item()
        level_loss += loss_level.item()
    
    train_loss = total_loss / len(train_loader)
    train_loss_epoch.append(train_loss)
    nk_losses= n_k_loss/len(train_loader)
    nk_loss_epoch.append(nk_losses)
    thick_losses = thick_loss / len(train_loader)
    thick_loss_epoch.append(thick_losses)
    level_losses = level_loss/len(train_loader)
    level_loss_epoch.append(level_losses)
    
    print ("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch+1, epochs, train_loss))
    print ("Epoch [{}/{}], nk Loss: {:.4f}".format(epoch+1, epochs, nk_losses))
    print ("Epoch [{}/{}], Thick Loss: {:.4f}".format(epoch+1, epochs, thick_losses))
    print("Epoch [{}/{}], level Loss: {:.4f}".format(epoch+1, epochs, level_losses))
    print('Train Time: ',time.time()-start)
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(tqdm(val_loader, desc='*********Evaluation mode*******')):
            pred1 = model(data['X'].float().to(device))
            loss_nk = loss_fn2(pred1[:,:142], data['Y'].float().to(device)[:,:142])
            loss_thick=loss_fn(pred1[:,142], data['Y'].float().to(device)[:,142])
            loss_level=loss_fn(pred1[:,143], data['Y'].float().to(device)[:,143])
        
            loss_val=torch.sqrt(loss_nk) + loss_thick + loss_level
            total_val_loss += loss_val.item()
            
    val_loss = total_val_loss / len(val_loader)
    val_loss_epoch.append(val_loss)
    print ("Epoch [{}/{}], Eval Loss: {:.4f}".format(epoch+1, epochs, val_loss))
    
    #Save best model
    if val_loss < n_val_loss:
        n_val_loss = val_loss
        torch.save(model.state_dict(), f'test_percent_unit_{version}_{lr}_{epochs}_MAE.pth')
        print("Best Model saved......")
print('Operation over:' ,time.time()-start)

torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
torch.Size([2048, 32, 36])
t

KeyboardInterrupt: 

Save_loss

In [8]:
train_loss_data=pd.DataFrame(train_loss_epoch)
train_loss_data.to_csv(f'weight_upper_train_loss_lr={lr}_epochs={epochs}_Unet.csv')
n_k_loss_data=pd.DataFrame(nk_loss_epoch)
n_k_loss_data.to_csv(f'weight_upper_n_k_loss_lr={lr}_epochs={epochs}_Unet.csv')
thick_loss_data=pd.DataFrame(thick_loss_epoch)
thick_loss_data.to_csv(f'weight_upper_thick_loss_lr={lr}_epochs={epochs}_Unet.csv')
level_loss_data=pd.DataFrame(level_loss_epoch)
level_loss_data.to_csv(f'weight_upper_level_loss_lr={lr}_epochs={epochs}_Unet.csv')
val_loss_data=pd.DataFrame(val_loss_epoch)
val_loss_data.to_csv(f'weight_upper_val_loss_lr={lr}_epochs={epochs}_Unet.csv')

Test_using_pre-trained_weight

In [15]:
test_file='R_test_weight.xlsx'
test=pd.read_excel(test_file)
test

,Unnamed: 0,1000,1020,1040,1060,1080,1100,1120,1140,1160,...,5300,5600,5900,6200,6500,6800,7100,7400,7700,8000
0,0,0.43274,0.45783,0.52730,0.59830,0.65145,0.67806,0.60875,0.53671,0.51018,...,0.12483,0.10819,0.09743,0.08981,0.08758,0.09001,0.09514,0.10468,0.11640,0.12928
1,1,0.43904,0.45938,0.52918,0.60372,0.66068,0.69203,0.62289,0.54710,0.51771,...,0.12187,0.10667,0.09560,0.08961,0.08885,0.09206,0.09896,0.10972,0.12254,0.13703
2,2,0.44766,0.47016,0.53749,0.60886,0.66225,0.68836,0.61643,0.54579,0.51949,...,0.12975,0.11030,0.09679,0.08965,0.08626,0.08614,0.09011,0.09838,0.10924,0.12265
3,3,0.44381,0.46831,0.53629,0.60635,0.65804,0.68132,0.60812,0.53913,0.51423,...,0.13201,0.11226,0.09741,0.09049,0.08637,0.08561,0.08862,0.09546,0.10493,0.11727
4,4,0.45495,0.47851,0.54659,0.61701,0.66936,0.69344,0.62036,0.55081,0.52515,...,0.13925,0.11865,0.10473,0.09669,0.09205,0.09209,0.09437,0.10121,0.11054,0.12158
5,5,0.44943,0.47328,0.54287,0.61628,0.67115,0.69813,0.62540,0.55229,0.52505,...,0.12488,0.10616,0.09400,0.08691,0.08540,0.08701,0.09278,0.10200,0.11422,0.12778


In [18]:
"""
모델 테스트
"""

test_model = ConvNet()

# test 파일 경로 및 test 데이터 로드
path_test = 'R_test_weight.xlsx'
#path_test='val_same_layer_level.csv'
class TestDataset(Dataset):
    def __init__(self, path_test):
        super(TestDataset, self).__init__()
        test = pd.read_excel(path_test)
        self.test_X = test.iloc[:,1:]
        self.tmp_x = self.test_X.values
    
    def __len__(self):
        return len(self.test_X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.tmp_x)[idx]
    
test_data = TestDataset(path_test)
test_loader = DataLoader(test_data, batch_size=6,pin_memory=True, num_workers=0)

In [19]:
# 모델에 학습된 가중치를 업로드.
weights = torch.load(f'test_percent_unit_{version}_{lr}_{epochs}_MAE.pth', map_location='cuda:1')
#weights = torch.load(f'test_percent_unit_(23, 36)_0.001_100_MSE+MAE.pth', map_location='cuda:1')
test_model.load_state_dict(weights)
test_model = test_model.to(device)
test_model.eval()
test_time=time.time()
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        outputs = test_model(data.float())
    print('Test Time : ',time.time()-test_time)
pred_test = outputs

Test Time :  0.0


In [20]:
#test_layer=[[str(i) for i in np.arange(1000,8100,100).tolist()],[str(i) for i in np.arange(1000,8100,100).tolist()],['layer(nm)','level(%)']]
test_layer=[[str(i) for i in wavenumbers.tolist()],[str(i) for i in wavenumbers.tolist()],['layer(nm)','level(%)']]
test_layer=list(chain(*test_layer))

In [21]:
sub=pred_test.cpu().numpy()
sub=pd.DataFrame(data=sub, columns=test_layer)
sub.to_csv('weight_upper_result_UNet(weight).csv',index='id')

In [22]:
sub #200102 12:21 두께만 안 맞음.

,1000.0,1020.0,1040.0,1060.0,1080.0,1100.0,1120.0,1140.0,1160.0,1180.0,...,5900.0,6200.0,6500.0,6800.0,7100.0,7400.0,7700.0,8000.0,layer(nm),level(%)
0,2.217420,2.337192,2.504164,2.767029,3.213556,4.865026,1.525314,0.929571,0.964791,0.675197,...,-2.546377e-07,0.000031,0.000036,0.000037,0.000453,0.000006,0.000004,0.000005,271.227386,0.006592
1,2.377083,2.512094,2.697872,3.001064,3.526159,5.187152,1.641496,0.903489,0.922200,0.681825,...,-1.441124e-05,0.000032,0.000040,0.000036,0.000350,0.000006,0.000006,0.000005,271.427490,0.649024
2,2.456431,2.596954,2.795885,3.116758,3.664362,5.102752,1.610471,0.864236,0.873421,0.657356,...,1.649195e-05,0.000038,0.000045,0.000038,0.000357,0.000006,0.000003,0.000005,254.559830,1.628768
3,2.345692,2.475457,2.655842,2.947276,3.455750,5.058508,1.564708,0.905999,0.920500,0.674646,...,1.714744e-05,0.000036,0.000045,0.000036,0.000391,0.000006,0.000004,0.000005,255.111160,0.941806
4,2.531913,2.683687,2.901067,3.253241,3.859861,5.188683,1.598520,0.815064,0.784694,0.535407,...,4.529405e-06,0.000040,0.000044,0.000038,0.000314,0.000006,0.000003,0.000005,253.473373,1.998475
5,2.455280,2.598201,2.802575,3.133687,3.722151,5.257517,1.524615,0.787010,0.764194,0.537101,...,7.469382e-07,0.000037,0.000044,0.000038,0.000346,0.000006,0.000003,0.000005,256.997955,1.654577


In [109]:
a=pd.read_csv(path_train)
a.head()

,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,...,7100.2,7200.2,7300.2,7400.2,7500.2,7600.2,7700.2,7800.2,7900.2,8000.2
0,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,...,0.146337,0.143524,0.140819,0.138209,0.135781,0.133682,0.131607,0.129836,0.128265,0.126791
1,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,...,0.166337,0.163524,0.160819,0.158209,0.155781,0.153682,0.151607,0.149836,0.148265,0.146791
2,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,...,0.186337,0.183524,0.180819,0.178209,0.175781,0.173682,0.171607,0.169836,0.168265,0.166791
3,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,...,0.206337,0.203524,0.200819,0.198209,0.195781,0.193682,0.191607,0.189836,0.188265,0.186791
4,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,1.224745,...,0.226337,0.223524,0.220819,0.218209,0.215781,0.213682,0.211607,0.209836,0.208265,0.206791


In [41]:
from torchviz import make_dot
import torch.onnx

In [42]:
custom=ConvNet()
params = custom.state_dict()
dummy_data = torch.empty(1, 71, dtype=torch.float32)
torch.onnx.export(custom, dummy_data, 'Unet+FCN.onnx')